In [1]:
import polars as pl

In [2]:
# Lista de colunas de datas para converter
date_cols = ['DT_NOTIFIC', 'DT_SIN_PRI', 'DT_INVEST', 'DT_ALRM', 'DT_GRAV']

# Lista de Sintomas e Comorbidades (Campos que geralmente são 1=Sim, 2=Não)
sintomas_cols = [
    'FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA', 'VOMITO', 'NAUSEA', 
    'DOR_COSTAS', 'CONJUNTVIT', 'ARTRITE', 'ARTRALGIA', 'PETEQUIA_N', 
    'LEUCOPENIA', 'DOR_RETRO'
]

comorbidades_cols = [
    'DIABETES', 'HEMATOLOG', 'HEPATOPAT', 'RENAL', 'HIPERTENSA', 
    'ACIDO_PEPT', 'AUTO_IMUNE'
]

sinais_alarme_cols = [
    'ALRM_HIPOT', 'ALRM_PLAQ', 'ALRM_VOM', 'ALRM_SANG', 'ALRM_HEMAT', 
    'ALRM_ABDOM', 'ALRM_LETAR', 'ALRM_HEPAT', 'ALRM_LIQ'
]

cols_binarias = [
    'ALRM_HIPOT', 'ALRM_PLAQ', 'ALRM_VOM', 'ALRM_SANG', 'ALRM_HEMAT', 
    'ALRM_ABDOM', 'ALRM_LETAR', 'ALRM_HEPAT', 'ALRM_LIQ',
    'GRAV_PULSO', 'GRAV_CONV', 'GRAV_ENCH', 'GRAV_INSUF', 'GRAV_TAQUI',
    'GRAV_EXTRE', 'GRAV_HIPOT', 'GRAV_HEMAT', 'GRAV_MELEN', 'GRAV_METRO',
    'GRAV_SANG', 'GRAV_AST', 'GRAV_MIOC', 'GRAV_CONSC', 'GRAV_ORGAO'
] + comorbidades_cols + sintomas_cols



In [3]:
df = pl.read_csv("./data/dengue_tratado.csv", null_values=['', ' ', 'nan'], ignore_errors=True)
df.shape

(9752880, 85)

In [4]:
cols_existentes = [c for c in cols_binarias  if c in df.columns]

In [5]:
df_clean = df.with_columns([
    pl.col(col).str.to_date(format="%Y-%m-%d", strict=False).alias(col)
    for col in df.columns if col.startswith("DT_")
]).with_columns([
    # Regra do SINAN: 
    # Começa com 4: Anos (4030 = 30 anos)
    # Começa com 3: Meses (3005 = 5 meses -> 0 anos)
    # Começa com 2: Dias  (2010 = 10 dias -> 0 anos)
    pl.col("NU_IDADE_N").cast(pl.Int32, strict=False),
    pl.when(pl.col("NU_IDADE_N") >= 4000)
    .then(pl.col("NU_IDADE_N") - 4000)
    .when(pl.col("NU_IDADE_N") >= 3000)
    .then(0)
    .when(pl.col("NU_IDADE_N") >= 2000)
    .then(0)
    .otherwise(None)
    .alias("IDADE_ANOS")
]).with_columns([
    pl.col("IDADE_ANOS").cut(
        breaks=[10, 20, 40, 60, 80],
        labels=["0-10", "11-20", "21-40", "41-60", "61-80", "80+"]
    ).cast(pl.Utf8).alias("FAIXA_ETARIA"),
    
    (pl.col("DT_INVEST") - pl.col("DT_SIN_PRI")).dt.total_days().fill_null(0).clip(lower_bound=0).alias("DIAS_SINTOMA_INVESTIGACAO"),
    (pl.col("DT_INVEST") - pl.col("DT_ALRM")).dt.total_days().fill_null(0).clip(lower_bound=0).alias("DIAS_ALARME_INVESTIGACAO"),
    (pl.col("DT_INVEST") - pl.col("DT_GRAV")).dt.total_days().fill_null(0).clip(lower_bound=0).alias("DIAS_GRAVE_INVESTIGACAO"),
    
    pl.when(pl.col("CS_GESTANT").is_in([1, 2, 3, 4]))
      .then(1)
      .otherwise(0)
      .alias("IS_GESTANTE"),
    pl.col("CS_SEXO").cast(pl.Categorical)
   
]).with_columns([
    pl.when(pl.col(c).cast(pl.Int32, strict=False) == 1).then(1).otherwise(0).alias(f"BIN_{c}")
    for c in cols_existentes
])

In [6]:
if 'df' in globals():
    del df

df_final = df_clean.with_columns([
    pl.when(pl.col("EVOLUCAO").is_in([2, 3]))
    .then(1)
    .otherwise(0)
    .alias("TARGET_OBITO")
])

if 'df_clean' in globals():
    del df_clean
  

In [7]:
  
cols_to_keep = [
    "TARGET_OBITO", "IDADE_ANOS", "FAIXA_ETARIA", "DIAS_SINTOMA_INVESTIGACAO", "DIAS_ALARME_INVESTIGACAO", "DIAS_GRAVE_INVESTIGACAO", "CS_SEXO", "SG_UF", "IS_GESTANTE"
] + [f"BIN_{c}" for c in cols_binarias  if c in df_final.columns]

df_model = df_final.select(cols_to_keep)

if 'df_final' in globals():
    del df_final

In [8]:
df_model['TARGET_OBITO'].value_counts()

TARGET_OBITO,count
i32,u32
1,12550
0,9740330


In [9]:
df_model.write_parquet("./data/dengue.parquet")